# Query Sentinel 1 scene from SHUB and load it with stackstac
- Query shub
- Get aws url
- Convert to stack Item
- Load with stackstac

In [15]:
from sentinelhub import CRS, BBox, DataCollection, SHConfig
from oxeo.core.data import get_aoi_from_s1_shub_catalog,get_aoi_from_stac_catalog

from sentinelhub import SHConfig

config = SHConfig()
config

SHConfig(
  instance_id='1bd87fe7-612b-4d12-bba1-392b5b7a435b',
  sh_client_id='99a83c8e-9ac1-481d-8c21-0f91c6f898cf',
  sh_client_secret='P#Z)wh3E?x+9EB^<63}uZh*W-qz-:;Ixn+FT~]Kf',
  sh_base_url='https://services.sentinel-hub.com',
  sh_auth_base_url='https://services.sentinel-hub.com',
  geopedia_wms_url='https://service.geopedia.world',
  geopedia_rest_url='https://www.geopedia.world/rest',
  aws_access_key_id='',
  aws_secret_access_key='',
  aws_session_token='',
  aws_metadata_url='https://roda.sentinel-hub.com',
  aws_s3_l1c_bucket='sentinel-s2-l1c',
  aws_s3_l2a_bucket='sentinel-s2-l2a',
  opensearch_url='http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2',
  max_wfs_records_per_query=100,
  max_opensearch_records_per_query=500,
  max_download_attempts=4,
  download_sleep_time=5.0,
  download_timeout_seconds=120.0,
  number_of_download_processes=1
)

In [17]:
from sentinelhub import SentinelHubCatalog

catalog = SentinelHubCatalog(config=config)


In [23]:
caspian_sea_bbox = BBox([49.9604, 44.7176, 51.0481, 45.2324], crs=CRS.WGS84)
time_interval = "2020-12-10", "2021-02-01"

search_params = {

}
s1_aoi = get_aoi_from_stac_catalog(catalog=catalog,
                                   data_collection=DataCollection.SENTINEL1,
                                   bbox=caspian_sea_bbox,
                                   time_interval=time_interval, search_params={})


datacollection.sentinel1


In [24]:
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import rescale_intensity
os.environ["AWS_REQUEST_PAYER"] = "requester"
img = s1_aoi.sel(band=["vh"])[:3].median(axis=0)[0,:1000,:1000].compute()
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))

2022-06-26 22:28:52,146 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,149 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,154 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,154 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,162 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,165 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,166 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/credentials
2022-06-26 22:28:52,167 |     INFO | credentials:load:1311 - Found credentials in shared credentials file: ~/.aws/cred

ValueError: could not broadcast input array from shape (0,1024) into shape (1024,1024)

# Query Sentinel 1 scene from ASF and load it with stackstac
- Query asf
- Convert asf granule to aws url
- Convert to stack Item
- Load with stackstac

In [ ]:
%load_ext autoreload

%autoreload 2


import requests
import os
from oxeo.core.data import query_asf, asf_granule_to_aws, CusotomSentinel1Reader
from oxeo.core.stac import sentinel1
import stackstac
os.environ["AWS_REQUEST_PAYER"] = "requester"

In [ ]:
item = sentinel1.create_item(results[0]["assets"]["s3"]["href"])

In [ ]:
stack = stackstac.stack(item)
stack

In [ ]:
import pyproj
bbox = [49.9604, 44.7176, 51.0481, 45.2324]

min_x_utm, min_y_utm = pyproj.Proj(stack.crs)(
        bbox[0], bbox[1]
    )
max_x_utm, max_y_utm = pyproj.Proj(stack.crs)(
        bbox[2], bbox[3]
    )

In [ ]:
aoi = stack.loc[..., max_y_utm:min_y_utm, min_x_utm:max_x_utm]
aoi

In [ ]:
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

geo: dict = {'type': 'Polygon',
   'coordinates': [
          [
            [
              -3.14208984375,
              37.470498470798724
            ],
            [
              -2.7960205078125,
              37.330856613297144
            ],
            [
              -2.61474609375,
              37.49229399862877
            ],
            [
              -2.6751708984375,
              37.6968609874419
            ],
            [
              -3.0267333984375,
              37.640334898059486
            ],
            [
              -3.14208984375,
              37.470498470798724
            ]
          ]]}
polygon = shape(geo)

In [ ]:
df = query_asf(platform="Sentinel-1A,Sentinel-1B",
               processing_level="GRD_HD",
               beam_mode="IW",
               intersects_with_wkt=polygon.wkt,
               start="2020-04-05T00:00:00UTC",
               end="2020-05-05T00:00:00UTC")

In [ ]:
asf_granule_to_aws(df.iloc[0])

In [ ]:
s3_url = asf_granule_to_aws(df.iloc[0])
item = sentinel1.create_item(s3_url)

In [ ]:

arr = stackstac.stack(item,reader=CusotomSentinel1Reader)